In [1]:
#pull in all the data across notebooks
%store -r HC_reltext
%store -r HC_alltext

%store -r IND_reltext
%store -r IND_alltext

%store -r Energy_reltext
%store -r Energy_alltext

%store -r CONSTA_reltext
%store -r CONSTA_alltext

%store -r CONDIS_reltext
%store -r CONDIS_alltext

%store -r IT_reltext
%store -r IT_alltext

%store -r Real_Estate_reltext
%store -r Real_Estate_alltext

%store -r Materials_reltext
%store -r Materials_alltext

%store -r Utilities_reltext
%store -r Utilities_alltext

In [2]:
# looping to reformat into a single list
all_relevant = HC_reltext+IND_reltext+Energy_reltext+CONSTA_reltext+CONDIS_reltext+IT_reltext+Real_Estate_reltext+Materials_reltext+Utilities_reltext

# all_relevant[0][0]

relevant = []

for company in all_relevant:

    for line in company:
        
        relevant.append(line)

len(relevant)

1603

In [3]:
# same as above but for all text
all_text = HC_alltext+IND_alltext+Energy_alltext+CONSTA_alltext+CONDIS_alltext+IT_alltext+Real_Estate_alltext+Materials_alltext+Utilities_alltext

all = []

for company in all_text:

    for line in company:

        all.append(line)

len(all)

115199

In [4]:
# pulling out the irrelevant text from all text
irrelevant = []

for i in all:
    if i not in relevant:
        irrelevant.append(i)

# print(irrelevant)

In [5]:
# creating a label/target list to classify the relevant text. 1 = relevant
class_rel = []

for i in relevant:
    class_rel.append(1)

len(class_rel)

1603

In [6]:
# same as above, for for irrelevant text. 0 = irrelevant
class_irr = []

for i in irrelevant:
    class_irr.append(0)

class_irr[0]

0

In [7]:
# creating dfs of irrelevant and relevant text
import pandas as pd

zipped = list(zip(relevant, class_rel))
zipped2 = list(zip(irrelevant, class_irr))

df = pd.DataFrame(zipped, columns=['text', 'class'])
df2 = pd.DataFrame(zipped2, columns = ['text', 'class'])


In [8]:
# concatenating the two dfs

merged = pd.concat([df, df2], ignore_index=True)

merged['class'].unique()

len(merged)

102350

In [9]:
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from nltk.stem import PorterStemmer
import re

stemmer = PorterStemmer()
words = stopwords.words("english")


In [10]:
# processing the text to take out extraneous words and stem any applicable words
merged['processed'] = merged['text'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

In [11]:
merged.head()

,text,class,processed
0,"From 2012 to 2020, we achieved a 26% reduction...",1,from achiev reduct absolut emiss
1,"In 2021, we achieved a 9% absolute emissions ...",1,in achiev absolut emiss reduct versu
2,This reduction was partially driven by energy...,1,thi reduct partial driven energi e cienci impr...
3,"In 2021, 9",1,in
4,6% of our purchased electricity came from rene...,1,purchas electr came renew sourc


In [12]:
merged['processed'].unique
merged.dropna(how='any')

,text,class,processed
0,"From 2012 to 2020, we achieved a 26% reduction...",1,from achiev reduct absolut emiss
1,"In 2021, we achieved a 9% absolute emissions ...",1,in achiev absolut emiss reduct versu
2,This reduction was partially driven by energy...,1,thi reduct partial driven energi e cienci impr...
3,"In 2021, 9",1,in
4,6% of our purchased electricity came from rene...,1,purchas electr came renew sourc
...,...,...,...
102345,\n→ NextEra Energy Resources would invest in e...,0,nextera energi resourc would invest electr com...
102346,\n→ All non-FPL fossil generation assets would...,0,all non fpl fossil gener asset would reach end...
102347,\n→ Vehicle fleet conversions are based on the...,0,vehicl fleet convers base avail clean fuel ele...
102348,Assumptions\n0\n0\nREALZERO,0,assumpt realzero


In [13]:
merged[['processed', 'class']].to_csv('data.csv')

In [14]:
# splitting into test train sets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
 

label = merged['class']

df = merged[['processed', 'class']]

#1603 1s
#100747 0s

#create dataset with equal number of relavent and irrelavant sentences

'''
dfrelavant = df.head(1603)
dfirrelavant = df.tail(100747).sample(n=1603)

dfbalanced = pd.concat([dfrelavant, dfirrelavant], ignore_index=True)
dfbalanced.to_csv('balanced_data.csv')

X_train, X_test, y_train, y_test = train_test_split(dfbalanced['processed'], dfbalanced['class'], test_size=0.1, random_state=100)

dfbalanced_train = pd.concat([X_train.to_frame(), y_train.to_frame()], axis = 1, ignore_index = True)
dfbalanced_train.columns = ['processed', 'class']
dfbalanced_train.to_csv('data/balanced_data_train.csv')

dfbalanced_test = pd.concat([X_test.to_frame(), y_test.to_frame()], axis = 1, ignore_index = True)
dfbalanced_test.columns = ['processed', 'class']
dfbalanced_test.to_csv('data/balanced_data_test.csv')



df_relavant_train = df.head(1442)
df_irrelavant_train = df.tail(90672)
df_train = pd.concat([df_relavant_train, df_irrelavant_train], ignore_index=True)
df_train.to_csv('data/labled_data_train.csv')

df_relavant_train = df.loc[1443:1604]
df_irrelavant_test = df.loc[1604:10075]
df_train = pd.concat([df_relavant_train, df_irrelavant_test], ignore_index=True)
df_train.to_csv('data/labled_data_test.csv')

'''



"\ndfrelavant = df.head(1603)\ndfirrelavant = df.tail(100747).sample(n=1603)\n\ndfbalanced = pd.concat([dfrelavant, dfirrelavant], ignore_index=True)\ndfbalanced.to_csv('balanced_data.csv')\n\nX_train, X_test, y_train, y_test = train_test_split(dfbalanced['processed'], dfbalanced['class'], test_size=0.1, random_state=100)\n\ndfbalanced_train = pd.concat([X_train.to_frame(), y_train.to_frame()], axis = 1, ignore_index = True)\ndfbalanced_train.columns = ['processed', 'class']\ndfbalanced_train.to_csv('data/balanced_data_train.csv')\n\ndfbalanced_test = pd.concat([X_test.to_frame(), y_test.to_frame()], axis = 1, ignore_index = True)\ndfbalanced_test.columns = ['processed', 'class']\ndfbalanced_test.to_csv('data/balanced_data_test.csv')\n\n\n\ndf_relavant_train = df.head(1442)\ndf_irrelavant_train = df.tail(90672)\ndf_train = pd.concat([df_relavant_train, df_irrelavant_train], ignore_index=True)\ndf_train.to_csv('data/labled_data_train.csv')\n\ndf_relavant_train = df.loc[1443:1604]\ndf_ir

In [15]:
print(X_train.values.astype('U'))

NameError: name 'X_train' is not defined

In [ ]:
from xml.sax.handler import feature_namespace_prefixes
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_df=0.7)

train_tfIdf = vectorizer_tfidf.fit_transform(X_train.values.astype('U'))

test_tfIdf = vectorizer_tfidf.transform(X_test.values.astype('U'))

# print(vectorizer_tfidf.get_feature_names()[5000:5010])

In [ ]:
# for val, feature_name in zip(vectorizer_tfidf.idf_, vectorizer_tfidf.get_feature_names_out()):
#     print(val, "and", feature_name)

In [ ]:
#naive bayes classifier prediction and evaluation
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report

nb_classifier = MultinomialNB()

nb_classifier.fit(train_tfIdf, y_train)

pred = nb_classifier.predict(test_tfIdf) 

cr = classification_report(pred, y_test)

print(cr)


In [ ]:
# logistic regression prediction and evaluation
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_tfIdf, y_train)

predLR =  lr.predict(test_tfIdf)

crLR = classification_report(pred, y_test)

print(crLR)
